# Will It Blend? Deploying BlenderBot

First, load the model.

In [1]:
from parlai.core.agents import create_agent_from_model_file
blender_agent = create_agent_from_model_file("blender_distilled/blender_distilled.pth.checkpoint",
                                            opt_overrides={
                                                "model": "internal:cgen",
                                                "skip_generation": False,
                                                "topk": 5,
                                                "inference": "topk",
                                                "temperature": 0.7,
                                            })



03:53:37 | Overriding opt["model"] to internal:cgen (previously: projects.anti_scaling.distillation:DistillTransformerAgent)
03:53:37 | Overriding opt["skip_generation"] to False (previously: True)
03:53:37 | Overriding opt["topk"] to 5 (previously: 10)
03:53:37 | Overriding opt["inference"] to topk (previously: beam)
03:53:37 | Overriding opt["temperature"] to 0.7 (previously: 1.0)
03:53:38 | Using CUDA
03:53:38 | loading dictionary from blender_distilled/blender_distilled.pth.checkpoint.dict
03:53:38 | num words = 8008
03:54:01 | Total parameters: 1,122,675,200 (1,122,019,840 trainable)
03:54:01 | Loading existing model params from blender_distilled/blender_distilled.pth.checkpoint


In [2]:
%load_ext autoreload
%autoreload 2

In [70]:
blender_agent.opt["inference"] = "delayedbeam"
blender_agent.opt["beam_delay"] = 25
blender_agent.opt["topp"] = 1.0
blender_agent.opt["topk"] = 5


## Example

Minimal example from Github issue

In [71]:

from itertools import zip_longest
import time
from parlai.utils.strings import normalize_reply

start = time.time()

blender_agent.reset()

history = [
    "hi, where are you from?",
    "i'm from the states! i'm from california originally but i live in utah now.",
    "cool, i am also from utah.",
    "that's great, what is your favorite place in utah?",
    "probably logan because that's where i'm from",
    "oh cool, that's a great city to live in. what do you do for a living?",
    "i am a student",
    "that's awesome, what are you studying?",
    "machine learning",
    "That's awesome! I'm currently studying to be a teacher. I hope to teach english one day.",
    "very nice my girlfriend is a teacher herself",
    "That's great! I hope she teaches english too. I love english. What grade level does she teach?",
    "fifth grade"
]
prefix = None
def process(history):
    pairs = zip_longest(*[iter(history)]*2)

    for user, bot in pairs:
        blender_agent.observe({'text': user, 'episode_done': False})
        if bot:
            blender_agent.history.add_reply(bot)

    response = blender_agent.act(prefix_text=prefix)
    elapsed = time.time() - start
    print(f"BlenderBot replied in {elapsed * 1000:.0f}ms: {normalize_reply(response['text'])}")
    print("All beams:", "\n".join([t for t, s in response["beam_texts"]]))
    print()

    print("BlenderBot's history view:")
    print(blender_agent.history)
    return response
    
response = process(history)

BlenderBot replied in 554ms: That's a good age. I'm sure she's a wonderful teacher. Do you have any hobbies?
All beams: That's a good age. I'm sure she's a wonderful teacher. Do you have any hobbies?
That's a wonderful age. I'm sure she will enjoy it! Do you have any hobbies?
That's really great! Do you have any hobbies? I love to read. Do you?
That's a good age. I'm sure it'll be great! How long have you been together?
oh, that is a great age to teach. I teach middle school english, which is kindergarten.
That's so cool! I've always wanted to teach middle school english! I think it's important to know how to read.
Wow! That's still very young. I think my girlfriend will be a great teacher. Do you like english?
That's so cool! I've always wanted to teach middle school english! I think it's important to know how to read!
That's so cool! I've always wanted to teach middle school english! I think it's important to know how to teach.
Oh cool! I teach kindergarten. I'm hoping to teach fifte

In [43]:
response

{'id': 'Cgen',
 'episode_done': False,
 'text': 'oh, i have never been. i have been to the eastern part of the state called kansas.',
 'beam_texts': [('oh, i have never been. i have been to the eastern part of the state called kansas.',
   -8.08232307434082),
  ('i love logan, i love the mountains. i would love to go back to utah one day.',
   -8.748612403869629),
  ("oh cool! i've never been there, i've been to california.  do you like it?",
   -12.69615650177002),
  ("that's a great choice. i've heard of it! i like the mountains there. do you?",
   -12.906228065490723),
  ("I have never been there, what part of the state is it in?  I know it's a great place for music, movies, and sports.",
   -13.11520767211914),
  ("I have never been there, what part of the state is it in?  I know it's a great place for music, movies, and sports!",
   -13.658272743225098),
  ('that is cool. i have never been there before. i would love to though. i like to travel.',
   -13.969677925109863),
  ("I hav

In [18]:
from itertools import zip_longest
history = [
    "hi, where are you from",
    "i'm from the states! i'm from california originally but i live in utah now.",
    "cool, i am also from utah",
    "that's great, what is your favorite place?",
    "um maybe taiwan?"
]
pairs = zip_longest(*[iter(history)]*2)
print(list(pairs))

[('hi, where are you from', "i'm from the states! i'm from california originally but i live in utah now."), ('cool, i am also from utah', "that's great, what is your favorite place?"), ('um maybe taiwan?', None)]


In [14]:
from parlai.agents.transformer.transformer import TransformerGeneratorAgent
